## 训练rnn模型识别0-9的数字
- 训练集有600组数据
- 每组数据有100对训练数据
- 每组数据为一个图片和一个正确的label
- 即输入为28-28的图片，输出为label-这个图片代表的数字

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# 超参数

In [3]:
# Hyper-parameters
sequence_length = 28    #序列长度，一个图片的高
input_size = 28         #输入大小，一个照片的宽
hidden_size = 128       #隐含层大小
num_layers = 2          #？
num_classes = 10        #分类个数，10分列，十个数字0-9
batch_size = 100        #每批的个数，每批100个数据，共600次
num_epochs = 2          #整体循环两次，即第一次按批次训练，之后重新在训练一次
learning_rate = 0.01    #学习率，梯度下降的调节参数

# 数据下载与加载

In [4]:
# MNIST dataset
#torchvision包 包含了目前流行的数据集，模型结构和常用的图片转换工具。
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# 数据分析

In [5]:
print(train_loader)
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ../../data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [7]:
#print(enumerate(train_loader))  #列举train_loader的元素
a=list(train_loader)  #数据转化为list

In [7]:
len(a)

600

In [8]:
a[599][1]  
#600组数据
#每组数据100个label（batch=100），每个label对应一个image，每个image对应28*28数据

tensor([ 5,  5,  6,  9,  1,  2,  3,  3,  6,  4,  6,  5,  7,  2,
         8,  1,  2,  7,  1,  3,  6,  1,  8,  5,  9,  3,  8,  0,
         1,  6,  2,  1,  2,  3,  2,  4,  1,  2,  7,  5,  0,  9,
         3,  3,  2,  3,  0,  7,  2,  8,  1,  1,  6,  7,  9,  8,
         2,  7,  6,  1,  7,  7,  3,  9,  1,  0,  7,  4,  9,  1,
         2,  3,  6,  3,  5,  8,  0,  8,  5,  3,  3,  4,  1,  0,
         9,  4,  6,  4,  6,  5,  0,  4,  2,  0,  2,  3,  8,  1,
         2,  3])

In [75]:
a[0][0].size()
#100个28*28

torch.Size([100, 1, 28, 28])

In [76]:
a[0][0][0].size()
#28*28

torch.Size([1, 28, 28])

In [32]:
a[0][0][0].size()

torch.Size([1, 28, 28])

In [77]:
#a[0][0][0].reshape(-1, sequence_length, input_size).to(device)

In [13]:
len(train_loader)

600

In [134]:
n=0
for i, (image, labels) in enumerate(train_loader):
    n+=1
    image=image
    images = image.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)

In [138]:
image.size()

torch.Size([100, 1, 28, 28])

In [137]:
images.size()

torch.Size([100, 28, 28])

In [36]:
print('有 %s 组数据'%n)
print('每组image的元素个数%s'%torch.numel(images))
print('image维度%s，'%'28*28')
print('每组有%s对数据,即batch=%s'%(torch.numel(labels),torch.numel(labels)))
print(labels)
print(images.size())
print(labels.size())
#784=28*28
#size of every image is 28*28, 每个image对应一个label
#有600组，每组有100对数据

有 600 组数据
每组image的元素个数78400
image维度28*28，
每组有100对数据,即batch=100
tensor([ 0,  8,  8,  5,  7,  3,  9,  4,  7,  2,  5,  8,  6,  4,
         3,  3,  6,  7,  2,  8,  0,  3,  6,  8,  3,  4,  9,  5,
         2,  2,  6,  0,  1,  4,  3,  0,  9,  4,  3,  3,  4,  3,
         4,  1,  7,  8,  2,  1,  1,  5,  2,  9,  5,  1,  7,  7,
         5,  7,  2,  0,  7,  9,  4,  5,  0,  8,  4,  3,  7,  8,
         2,  3,  1,  3,  1,  1,  1,  8,  0,  7,  5,  3,  1,  9,
         4,  6,  4,  3,  1,  9,  6,  4,  1,  8,  1,  6,  7,  1,
         0,  7])
torch.Size([100, 28, 28])
torch.Size([100])


# rnn模型

In [17]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidde time step n state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [80]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
model

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

- lstm，输入为28（图片长），输出为128
- fc，输入为128，输出为10

# 模型分解

## 输入

In [79]:
images.size()  # x

torch.Size([100, 28, 28])

## 隐层

In [85]:
h0 = torch.zeros(num_layers, images.size(0),hidden_size).to(device)
c0 = torch.zeros(num_layers, images.size(0),hidden_size).to(device)

In [86]:
h0.size()  #128为隐层大小，100为一批输入的数据量，100个图片

torch.Size([2, 100, 128])

## 长短时记忆模型，一个图片通过28次输入获得最后输出；每次输入获得一次输出，最后的输出才为整个图片的输出

In [87]:
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
out, _ = lstm(images, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

In [91]:
print(out.size())  #out为三维，中间为第三维
print(out[:, -1, :].size())

torch.Size([100, 28, 128])
torch.Size([100, 128])


#### out[:, -1, :].size()---->选取out第28个输出（-1），因为图片是28行一行一行输入的，最后一行代表整个图片

In [101]:
#out[:, 27, :]=out[:, -1, :]

In [69]:
fc = nn.Linear(hidden_size, num_classes)
out = fc(out[:, -1, :])

In [70]:
out.size()

torch.Size([100, 10])

In [20]:
images = images.reshape(-1, sequence_length, input_size).to(device)

In [126]:
outputs = model.forward(images)   #=outputs = model(images)

In [127]:
outputs.size()

torch.Size([100, 10])

In [130]:
output1=outputs[:,0:10]

In [131]:
output1.size()

torch.Size([100, 10])

In [132]:
labels.size()

torch.Size([100])

In [143]:
loss = criterion(output1, labels)
loss

tensor(2.3089)

In [144]:
# Backward and optimize
optimizer.zero_grad()
loss.backward()
optimizer.step()

# Loss and optimizer

In [105]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()   #多分类损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [114]:
# Train the model
total_step = len(train_loader)   #600
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        #通过forward函数获得输出，输出与labels获得loss
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.6053
Epoch [1/2], Step [200/600], Loss: 0.2521
Epoch [1/2], Step [300/600], Loss: 0.1476
Epoch [1/2], Step [400/600], Loss: 0.1555
Epoch [1/2], Step [500/600], Loss: 0.1052
Epoch [1/2], Step [600/600], Loss: 0.1676
Epoch [2/2], Step [100/600], Loss: 0.1140
Epoch [2/2], Step [200/600], Loss: 0.0989
Epoch [2/2], Step [300/600], Loss: 0.1122
Epoch [2/2], Step [400/600], Loss: 0.0557
Epoch [2/2], Step [500/600], Loss: 0.0405
Epoch [2/2], Step [600/600], Loss: 0.0558


In [151]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 97.67 %
